In [ ]:
%pip install -i https://test.pypi.org/simple/ cryptogame==0.0.7

In [ ]:
from cryptogame.game.ufcma import *
from cryptogame.tools.utils import *
from cryptogame.tools.block_cipher import *
from simulate import *
from cryptogame.tools.modes_of_operation import CBC_MAC

In [ ]:
key_len = 10
msg_len = 10
B = BlockCipher(key_len, block_len=msg_len)

def cbc_mac(key, M):
	''' No padding'''
	c_0 = '0'*msg_len
	m = len(M) // msg_len
	assert len(M) == m*msg_len
	for i in range(m):
		c_i = B.evaluate(key, xor(c_0, M[i * msg_len : (i + 1) * msg_len]))
		c_0 = c_i
	return c_i

game = UFCMA(cbc_mac, key_len, 5)
game._initialize()

def adv_cbc_mac(game):
	'''Splicing Attack'''
	x = random_bits(msg_len)
	t1 = game.Tag(x)
	M = x + xor(t1, x)
	return (M, t1)

msg, tag = adv_cbc_mac(game)
game.finalize(msg, tag)

s = Simulate(game, adv_cbc_mac)
s.simulate_ufcma(verbose=True)

### Homework 4 Problem 2


In [ ]:
key_len = 10
block_len = 10
blockcipher = BlockCipher(key_len, block_len)

def mac(key, msg):
	cbc_mac = CBC_MAC(blockcipher)
	t = cbc_mac.Tag_gen(key, msg)
	T = blockcipher.evaluate(key, t)
	return T

game = UFCMA(mac, key_len, 20)

game._initialize()

In [ ]:
def adv_mac(game:UFCMA):
	x = random_bits(block_len)
	tag = game.Tag(x)
	m = x+'0'*block_len+xor(tag, x)
	return m, tag

forged_message, tag = adv_mac(game)

game.finalize(forged_message, tag)

In [ ]:
s = Simulate(game, adv_mac)
s.simulate_ufcma(verbose=True)